In [1]:
import pandas as pd

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32', 'onpromotion':'bool' }

# load or create your dataset
print('Load data...')
df_train = pd.read_csv("data/train.csv", dtype=dtypes, parse_dates=["date"], low_memory=True, usecols=[1, 2, 3, 4, 5], skiprows=range(1, 106458909) )
df_test = pd.read_csv("data/test.csv", usecols=[0, 1, 2, 3, 4], dtype={'onpromotion': bool}, parse_dates=["date"] )

Load data...


In [21]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import time

# load or create your dataset
print('Loading data...')

start = time.time()

train_list = []

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32', 'onpromotion':'bool' }
chunk_iter = pd.read_csv("data/train.csv", dtype=dtypes, parse_dates=["date"], low_memory=True, usecols=[1, 2, 3, 4, 5], chunksize=10000 ) #,converters=converters) 

for chunk in chunk_iter:
    chunk_kept = chunk.loc[chunk.date>=date(2017, 1, 1)] 
    train_list.append(chunk_kept) 
df_train = pd.concat(train_list)
df_test = pd.read_csv("data/test.csv", usecols=[0, 1, 2, 3, 4], dtype={'onpromotion': bool}, parse_dates=["date"] ).set_index(['store_nbr', 'item_nbr', 'date'])
items = pd.read_csv("data/items.csv").set_index("item_nbr")

print('Load complete...')
print("Time: ", time.strftime("%M:%S", time.gmtime(time.time() - start)) )

Loading data...
Load complete...
Time:  03:30


In [20]:
# Definimos la Ventana Temporal
look_back = 11
# Dias que se quieren predecir
days_to_predict = 16
# Fecha base
base_date = date(2017, 5, 31)
# Última fecha disponible
last_date = date(2017, 8, 16)

if (base_date + timedelta(days=7 * look_back) < last_date):
    print("Parámetros válidos: OK")
    print("Fecha máxima: ", base_date + timedelta(days=7 * look_back))
else:
    print("Fecha excede límite de registros")

Fecha excede límite de registros


In [4]:
df_train.loc[(df_train.unit_sales < 0),'unit_sales'] = 0 # Eliminar Valores Negativos

In [7]:
pd.datetime(2017,1,1)

datetime.datetime(2017, 1, 1, 0, 0)

In [8]:
date(2017, 1, 1)

datetime.date(2017, 1, 1)

In [5]:
df_train["unit_sales"] = df_train["unit_sales"].apply(np.log1p) # Aplicar Logaritmo

In [6]:
df_train

,date,store_nbr,item_nbr,unit_sales,onpromotion
101688779,2017-01-01,25,99197,0.693147,False
101688780,2017-01-01,25,103665,2.079442,False
101688781,2017-01-01,25,105574,0.693147,False
101688782,2017-01-01,25,105857,1.609438,False
101688783,2017-01-01,25,106716,1.098612,False
101688784,2017-01-01,25,108698,1.098612,False
101688785,2017-01-01,25,108786,0.693147,False
101688786,2017-01-01,25,108797,0.693147,False
101688787,2017-01-01,25,108862,0.693147,False
101688788,2017-01-01,25,108952,1.098612,False


In [16]:
df_train["date"] = df_train["date"].dt.dayofweek
df_test["date"] = df_test["date"].dt.dayofweek
df_train["unit_sales"] = df_train["unit_sales"].apply(np.log1p)

In [19]:
from sklearn.model_selection import train_test_split

X = df_train.drop("unit_sales", axis=1)
y = df_train["unit_sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [20]:
# create dataset for lightgbm
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [22]:
from sklearn.metrics import mean_squared_error
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Save model...')
# save model to file
gbm.save_model('model.txt')

print('Start predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Start training...
[1]	valid_0's l2: 577.803	valid_0's auc: 0.576309
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 575.924	valid_0's auc: 0.561389
[3]	valid_0's l2: 574.511	valid_0's auc: 0.538277
[4]	valid_0's l2: 573.237	valid_0's auc: 0.540839
[5]	valid_0's l2: 571.636	valid_0's auc: 0.540387
[6]	valid_0's l2: 570.419	valid_0's auc: 0.558021
Early stopping, best iteration is:
[1]	valid_0's l2: 577.803	valid_0's auc: 0.576309
Save model...
Start predicting...
The rmse of prediction is: 24.0375393188


In [27]:
# predict
y_pred_real = gbm.predict(, num_iteration=gbm.best_iteration)

In [29]:
len(y_pred_real)

3370464

In [25]:
y_pred_real

array([ 8.35974134,  8.35974134,  8.35974134, ...,  8.35974134,
        8.35974134,  8.35974134])

In [1]:
"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding more
average features and weekly average features on it.
"""
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32', 'onpromotion':'bool' }

# load or create your dataset
print('Loading data...')
df_train = pd.read_csv("data/train.csv", dtype=dtypes, parse_dates=["date"], low_memory=True, usecols=[1, 2, 3, 4, 5], skiprows=range(1, 106458909) )
df_test = pd.read_csv("data/test.csv", usecols=[0, 1, 2, 3, 4], dtype={'onpromotion': bool}, parse_dates=["date"] ).set_index(['store_nbr', 'item_nbr', 'date'])
items = pd.read_csv("data/items.csv").set_index("item_nbr")
print('Load complete...')

Loading data...
Load complete...


In [2]:
df_2017 = df_train
del df_train

In [3]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2017-02-16,30,1696003,1.224,False
1,2017-02-16,30,1696008,8.922,False
2,2017-02-16,30,1696013,0.333,False
3,2017-02-16,30,1696014,1.000,False
4,2017-02-16,30,1696025,3.000,False


In [4]:
# Apilar en columnas las filas de Date para Train enfocandose en las Promociones
promo_2017_train = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
# Asignar el nombre de las fechas en las columnas de Train
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_train.head()

date                2017-02-16  2017-02-17  2017-02-18  2017-02-19  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-02-20  2017-02-21  2017-02-22  2017-02-23  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-02-24  2017-02-25     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   
          103520         False       False     ...           False   
          103665         False       False     ...           False   
          105574         False       False     ...           False   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  
          103520         False       False       False       False       False  
          103665         False       False       False       False       False  
          105574         False       False       False       False       False  

[5 rows x 181 columns]

In [5]:
# Apilar en columnas las filas de Date para Test enfocandose en las Promociones
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.head()

onpromotion                                              \
date                2017-08-16 2017-08-17 2017-08-18 2017-08-19 2017-08-20   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103501         False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   

                                                                           \
date               2017-08-21 2017-08-22 2017-08-23 2017-08-24 2017-08-25   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                                                                           \
date               2017-08-26 2017-08-27 2017-08-28 2017-08-29 2017-08-30   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                               
date               2017-08-31  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103501        False  
          103520        False  
          103665        False

In [6]:
# Asignar el nombre de las fechas en las columnas de Test
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test.head()

date                2017-08-16  2017-08-17  2017-08-18  2017-08-19  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103501         False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   

date                2017-08-20  2017-08-21  2017-08-22  2017-08-23  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103501         False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103501         False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          103501         False       False       False       False  
          103520         False       False       False       False  
          103665         False       False       False       False

In [7]:
# Ajustamos los indices de Test en base a los de train
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017_test.head()

date                2017-08-16  2017-08-17  2017-08-18  2017-08-19  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-20  2017-08-21  2017-08-22  2017-08-23  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          103520         False       False       False       False  
          103665         False       False       False       False  
          105574         False       False       False       False

In [8]:
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [9]:
promo_2017.head()

date                2017-02-16  2017-02-17  2017-02-18  2017-02-19  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-02-20  2017-02-21  2017-02-22  2017-02-23  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-02-24  2017-02-25     ...      2017-08-22  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   
          103520         False       False     ...           False   
          103665         False       False     ...           False   
          105574         False       False     ...           False   

date                2017-08-23  2017-08-24  2017-08-25  2017-08-26  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-27  2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  
          103520         False       False       False       False       False  
          103665         False       False       False       False       False  
          105574         False       False       False       False       False  

[5 rows x 197 columns]

In [10]:
# Apilar en columnas las filas de Date para Train enfocandose en las Unidades
df_2017 = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)
# Asignar el nombre de las fechas en las columnas de Train
df_2017.columns = df_2017.columns.get_level_values(1)

In [45]:
#Ventas diarias (Columnas) de Unidades para cada Tienda y cada Item de tienda (Filas)
df_2017

date                2017-02-16  2017-02-17  2017-02-18  2017-02-19  \
store_nbr item_nbr                                                   
1         96995            0.0         0.0         0.0         0.0   
          99197            0.0         0.0         0.0         0.0   
          103520           0.0         1.0         0.0         0.0   
          103665           0.0         3.0         6.0         2.0   
          105574           0.0         6.0         7.0         0.0   
          105575           0.0        10.0        16.0         2.0   
          105577           0.0         2.0         0.0         0.0   
          105693           0.0         1.0         1.0         0.0   
          105737           0.0         3.0         0.0         5.0   
          105857           0.0         3.0         3.0         1.0   
          106716           0.0         0.0         2.0         0.0   
          108079           0.0         1.0         1.0         1.0   
          108634           0.0         0.0         0.0         0.0   
          108696           0.0         3.0         2.0         0.0   
          108698           0.0         5.0         2.0         1.0   
          108701           0.0         3.0         0.0         1.0   
          108786           0.0         3.0         2.0         0.0   
          108797           0.0         0.0         5.0         0.0   
          108862           0.0         0.0         0.0         0.0   
          108952           0.0         2.0         1.0         1.0   
          111223           0.0        14.0         9.0         3.0   
          111397           0.0         1.0         4.0         0.0   
          112830           0.0         0.0         0.0         1.0   
          114778           0.0         3.0        10.0         1.0   
          114790           0.0         6.0         5.0         3.0   
          114799           0.0         0.0         2.0         0.0   
          114800           0.0         6.0         5.0         2.0   
          115267           0.0         5.0         5.0         0.0   
          115611           0.0         5.0         3.0         0.0   
          115693           0.0         4.0         3.0         0.0   
...                        ...         ...         ...         ...   
54        2081056          0.0         0.0         0.0         0.0   
          2081064          0.0         0.0         0.0         0.0   
          2081095          0.0         0.0         0.0         0.0   
          2081142          0.0         0.0         0.0         0.0   
          2081161          0.0         0.0         0.0         0.0   
          2083837          0.0         0.0         0.0         0.0   
          2083859          0.0         0.0         0.0         0.0   
          2084278          0.0         0.0         0.0         0.0   
          2084557          0.0         0.0         0.0         0.0   
          2086882          0.0         0.0         0.0         0.0   
          2087374          0.0         0.0         0.0         0.0   
          2087409          0.0         0.0         0.0         0.0   
          2087543          0.0         0.0         0.0         0.0   
          2087567          0.0         0.0         0.0         0.0   
          2087933          0.0         0.0         0.0         0.0   
          2087978          0.0         0.0         0.0         0.0   
          2088922          0.0         0.0         0.0         0.0   
          2089036          0.0         0.0         0.0         0.0   
          2089339          0.0         0.0         0.0         0.0   
          2099230          0.0         0.0         0.0         0.0   
          2101795          0.0         0.0         0.0         0.0   
          2103250          0.0         0.0         0.0         0.0   
          2105347          0.0         0.0         0.0         0.0   
          2106464          0.0         0.0         0.0         0.0   
          

In [11]:
# Reindexo items en base a los Items disponibles en Train
items = items.reindex(df_2017.index.get_level_values(1))
items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1
105574,GROCERY I,1045,0


In [12]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        # Calcula el promedio de los 3 días anteriores a la fecha deseada t2017
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        #"mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        #"promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        #Promedio de Ventas en base a Dia de la Semana
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        #X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        # Unidades Vendidas de Items por tienda para los 16 días posteriores a la fecha deseada t2017
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [14]:
print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []

Preparing dataset...


In [15]:
print("Fecha base: ",t2017)
print("Fecha calculada: ",(t2017 + timedelta(days=7 * 1)))
# Calcula el promedio de los 3 días anteriores a la fecha deseada t2017
df_2017[pd.date_range((t2017 + timedelta(days=7 * 1)) - timedelta(days=3), periods=3, freq='D')]

Fecha base:  2017-05-31
Fecha calculada:  2017-06-07


date                2017-06-04  2017-06-05  2017-06-06
store_nbr item_nbr                                    
1         96995            0.0         0.0         0.0
          99197            2.0         2.0         0.0
          103520           0.0         1.0         4.0
          103665           2.0         3.0         5.0
          105574           3.0         3.0         7.0
          105575           8.0         6.0        15.0
          105577           0.0         2.0         1.0
          105693           0.0         3.0         0.0
          105737           0.0         3.0         2.0
          105857           0.0         5.0        12.0
          106716           1.0         4.0         3.0
          108079           0.0         1.0         0.0
          108634           0.0         0.0         0.0
          108696           0.0         4.0         2.0
          108698           2.0         4.0         1.0
          108701           1.0         2.0         0.0
          108786           3.0         1.0         2.0
          108797           0.0         1.0         0.0
          108862           1.0         0.0         1.0
          108952           0.0         1.0         2.0
          111223           3.0         7.0         5.0
          111397           0.0         0.0         1.0
          112830           0.0         4.0         3.0
          114778           1.0         0.0         1.0
          114790           1.0         3.0         8.0
          114799           1.0         3.0         2.0
          114800           1.0         5.0         3.0
          115267           0.0         0.0         2.0
          115611           3.0         3.0         1.0
          115693           2.0         4.0         2.0
...                        ...         ...         ...
54        2081056          9.0         2.0         4.0
          2081064         55.0        16.0        62.0
          2081095          0.0         8.0         4.0
          2081142          0.0         0.0         0.0
          2081161         61.0        13.0        26.0
          2083837          0.0         2.0         0.0
          2083859          4.0         0.0         0.0
          2084278          1.0         3.0         4.0
          2084557         52.0       780.0        71.0
          2086882          0.0         4.0         2.0
          2087374          0.0         1.0         4.0
          2087409          0.0         0.0         0.0
          2087543          0.0         0.0         0.0
          2087567          0.0         0.0         0.0
          2087933          1.0         0.0         2.0
          2087978          8.0         7.0         2.0
          2088922         18.0         7.0         5.0
          2089036          0.0         0.0         0.0
          2089339          5.0         6.0         2.0
          2099230          0.0         0.0         0.0
          2101795          0.0         0.0         0.0
          2103250          0.0         0.0         0.0
          2105347          3.0         1.0         1.0
          2106464          0.0         0.0         0.0
          2108569          0.0         0.0         0.0
          2109909          0.0         0.0         0.0
          2110456          0.0         0.0         0.0
          2113343          0.0         0.0         0.0
          2113914          0.0         0.0         0.0
          2116416          0.0         0.0         0.0

[165464 rows x 3 columns]

In [16]:
prepare_dataset(t2017 + timedelta(days=7 * 0))

(        day_1_2017  mean_14_2017  mean_30_2017  mean_3_2017  mean_60_2017  \
 0              0.0      0.214286      0.200000     0.333333      0.266667   
 1              0.0      0.928571      0.666667     1.000000      0.333333   
 2              0.0      1.500000      2.100000     0.000000      1.850000   
 3              0.0      2.142857      2.233333     0.666667      2.233333   
 4              2.0      5.000000      6.233333     2.000000      6.583333   
 5              9.0      7.714286      9.133333     6.000000      8.866667   
 6              0.0      0.714286      1.266667     1.000000      1.250000   
 7              0.0      0.000000      0.000000     0.000000      0.066667   
 8              1.0      2.642857      2.200000     2.333333      1.950000   
 9              5.0      6.642857      5.666667     4.666667      5.183333   
 10             4.0      1.857143      1.966667     2.000000      1.883333   
 11             0.0      1.000000      1.200000     0.000000    

In [17]:
# Elige una ventana temporal de 6
for i in range(6):
    delta = timedelta(days=7 * i)
    print("Calculando promedios deseados para la fecha: ",t2017 + delta)
    X_tmp, y_tmp = prepare_dataset(
        # Calcula promedios deseados para fechas cada 7 días (7,14,21,...,42)
        t2017 + delta
    )
    # Unir a lista los valores de X
    X_l.append(X_tmp)
    # Unir a lista los valores de y
    y_l.append(y_tmp)

# Concatenamos todos los valores de X
X_train = pd.concat(X_l, axis=0)
# Concatenamos todos los valores de y
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

Calculando promedios deseados para la fecha:  2017-05-31
Calculando promedios deseados para la fecha:  2017-06-07
Calculando promedios deseados para la fecha:  2017-06-14
Calculando promedios deseados para la fecha:  2017-06-21
Calculando promedios deseados para la fecha:  2017-06-28
Calculando promedios deseados para la fecha:  2017-07-05


In [18]:
# Train son los promedios de Unidades Vendidas, Promociones, Días de Semana de los últimos 7,14,30,60 días
X_train

,day_1_2017,mean_14_2017,mean_30_2017,mean_3_2017,mean_60_2017,mean_7_2017,promo_14_2017,promo_60_2017,mean_4_dow0_2017,mean_4_dow1_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,0.0,0.214286,0.200000,0.333333,0.266667,0.428571,0,0,0.00,0.25,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.928571,0.666667,1.000000,0.333333,1.142857,0,0,1.25,1.25,...,0,0,0,0,0,0,0,0,0,0
2,0.0,1.500000,2.100000,0.000000,1.850000,2.142857,0,0,3.50,4.00,...,0,0,0,0,0,0,0,0,0,0
3,0.0,2.142857,2.233333,0.666667,2.233333,1.571429,0,0,0.25,2.50,...,0,0,0,0,0,0,0,0,0,0
4,2.0,5.000000,6.233333,2.000000,6.583333,3.428571,0,21,9.00,5.75,...,0,0,0,0,0,0,0,0,0,0
5,9.0,7.714286,9.133333,6.000000,8.866667,8.142858,0,0,11.25,8.25,...,0,0,0,0,0,0,0,0,0,0
6,0.0,0.714286,1.266667,1.000000,1.250000,0.857143,0,0,1.50,1.50,...,0,0,0,0,0,0,0,0,0,0
7,0.0,0.000000,0.000000,0.000000,0.066667,0.000000,0,0,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
8,1.0,2.642857,2.200000,2.333333,1.950000,3.428571,0,0,6.00,0.75,...,0,0,0,0,0,0,0,0,0,0
9,5.0,6.642857,5.666667,4.666667,5.183333,5.000000,0,1,8.50,4.50,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Y_train (Unidades Vendidas de Items por tienda para los 16 días posteriores a la fecha deseada t2017)
df_2017[pd.date_range(t2017 + timedelta(days=7 * 0), periods=16)]

date                2017-05-31  2017-06-01  2017-06-02  2017-06-03  \
store_nbr item_nbr                                                   
1         96995            1.0         0.0         0.0         0.0   
          99197            1.0         3.0         2.0         6.0   
          103520           3.0         2.0         2.0         1.0   
          103665           8.0         0.0         5.0         5.0   
          105574           3.0        11.0         5.0         3.0   
          105575           5.0         9.0        14.0        10.0   
          105577           4.0         2.0         5.0         1.0   
          105693           0.0         0.0         0.0         0.0   
          105737           2.0         2.0         2.0         1.0   
          105857           8.0         8.0         7.0         0.0   
          106716           4.0         0.0         3.0         2.0   
          108079           1.0         1.0         0.0         2.0   
          108634           0.0         0.0         1.0         2.0   
          108696           5.0         2.0         4.0         3.0   
          108698           4.0         3.0         3.0         0.0   
          108701           3.0         0.0         4.0         3.0   
          108786           4.0         3.0         5.0         0.0   
          108797           1.0         3.0         2.0         4.0   
          108862           0.0         0.0         1.0         0.0   
          108952           1.0         0.0         1.0         4.0   
          111223          11.0         3.0         3.0        15.0   
          111397           1.0         1.0         1.0         1.0   
          112830           2.0         1.0         0.0         2.0   
          114778           1.0         0.0         5.0         5.0   
          114790           5.0         1.0         5.0         5.0   
          114799           0.0         1.0         3.0         2.0   
          114800           5.0         3.0         4.0         0.0   
          115267           0.0         2.0         4.0         2.0   
          115611           7.0         6.0         6.0        10.0   
          115693           5.0         0.0         1.0         6.0   
...                        ...         ...         ...         ...   
54        2081056          4.0         2.0         2.0         2.0   
          2081064         94.0        82.0        45.0        37.0   
          2081095          3.0         2.0         4.0         9.0   
          2081142          4.0         0.0         2.0         0.0   
          2081161         17.0         0.0        13.0        13.0   
          2083837          6.0         0.0         0.0         0.0   
          2083859          3.0         2.0         0.0         2.0   
          2084278          0.0         0.0         5.0         0.0   
          2084557          7.0         0.0       275.0        43.0   
          2086882          2.0         2.0         3.0         0.0   
          2087374          0.0         0.0         0.0         0.0   
          2087409          0.0         0.0         0.0         0.0   
          2087543          0.0         0.0         0.0         0.0   
          2087567          0.0         0.0         0.0         0.0   
          2087933          0.0         0.0         1.0         1.0   
          2087978          5.0         1.0         8.0         1.0   
          2088922         11.0         9.0        12.0        15.0   
          2089036          0.0         0.0         0.0         0.0   
          2089339          9.0         2.0         9.0         1.0   
          2099230          0.0         0.0         0.0         2.0   
          2101795          0.0         0.0         1.0         0.0   
          2103250          0.0         0.0         0.0         0.0   
          2105347          1.0         1.0         1.0         2.0   
          2106464          0.0         0.0         0.0         0.0   
          

In [20]:
# Calcula un X y y en base a una fecha deseada (Aún no comprendo porqué)
X_val, y_val = prepare_dataset(date(2017, 7, 26))
# Calculamos el X_test en base al último día + 1 disponible de registros
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

In [21]:
df_2017

date                2017-02-16  2017-02-17  2017-02-18  2017-02-19  \
store_nbr item_nbr                                                   
1         96995            0.0         0.0         0.0         0.0   
          99197            0.0         0.0         0.0         0.0   
          103520           0.0         1.0         0.0         0.0   
          103665           0.0         3.0         6.0         2.0   
          105574           0.0         6.0         7.0         0.0   
          105575           0.0        10.0        16.0         2.0   
          105577           0.0         2.0         0.0         0.0   
          105693           0.0         1.0         1.0         0.0   
          105737           0.0         3.0         0.0         5.0   
          105857           0.0         3.0         3.0         1.0   
          106716           0.0         0.0         2.0         0.0   
          108079           0.0         1.0         1.0         1.0   
          108634           0.0         0.0         0.0         0.0   
          108696           0.0         3.0         2.0         0.0   
          108698           0.0         5.0         2.0         1.0   
          108701           0.0         3.0         0.0         1.0   
          108786           0.0         3.0         2.0         0.0   
          108797           0.0         0.0         5.0         0.0   
          108862           0.0         0.0         0.0         0.0   
          108952           0.0         2.0         1.0         1.0   
          111223           0.0        14.0         9.0         3.0   
          111397           0.0         1.0         4.0         0.0   
          112830           0.0         0.0         0.0         1.0   
          114778           0.0         3.0        10.0         1.0   
          114790           0.0         6.0         5.0         3.0   
          114799           0.0         0.0         2.0         0.0   
          114800           0.0         6.0         5.0         2.0   
          115267           0.0         5.0         5.0         0.0   
          115611           0.0         5.0         3.0         0.0   
          115693           0.0         4.0         3.0         0.0   
...                        ...         ...         ...         ...   
54        2081056          0.0         0.0         0.0         0.0   
          2081064          0.0         0.0         0.0         0.0   
          2081095          0.0         0.0         0.0         0.0   
          2081142          0.0         0.0         0.0         0.0   
          2081161          0.0         0.0         0.0         0.0   
          2083837          0.0         0.0         0.0         0.0   
          2083859          0.0         0.0         0.0         0.0   
          2084278          0.0         0.0         0.0         0.0   
          2084557          0.0         0.0         0.0         0.0   
          2086882          0.0         0.0         0.0         0.0   
          2087374          0.0         0.0         0.0         0.0   
          2087409          0.0         0.0         0.0         0.0   
          2087543          0.0         0.0         0.0         0.0   
          2087567          0.0         0.0         0.0         0.0   
          2087933          0.0         0.0         0.0         0.0   
          2087978          0.0         0.0         0.0         0.0   
          2088922          0.0         0.0         0.0         0.0   
          2089036          0.0         0.0         0.0         0.0   
          2089339          0.0         0.0         0.0         0.0   
          2099230          0.0         0.0         0.0         0.0   
          2101795          0.0         0.0         0.0         0.0   
          2103250          0.0         0.0         0.0         0.0   
          2105347          0.0         0.0         0.0         0.0   
          2106464          0.0         0.0         0.0         0.0   
          

In [22]:
X_train

,day_1_2017,mean_14_2017,mean_30_2017,mean_3_2017,mean_60_2017,mean_7_2017,promo_14_2017,promo_60_2017,mean_4_dow0_2017,mean_4_dow1_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,0.0,0.214286,0.200000,0.333333,0.266667,0.428571,0,0,0.00,0.25,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.928571,0.666667,1.000000,0.333333,1.142857,0,0,1.25,1.25,...,0,0,0,0,0,0,0,0,0,0
2,0.0,1.500000,2.100000,0.000000,1.850000,2.142857,0,0,3.50,4.00,...,0,0,0,0,0,0,0,0,0,0
3,0.0,2.142857,2.233333,0.666667,2.233333,1.571429,0,0,0.25,2.50,...,0,0,0,0,0,0,0,0,0,0
4,2.0,5.000000,6.233333,2.000000,6.583333,3.428571,0,21,9.00,5.75,...,0,0,0,0,0,0,0,0,0,0
5,9.0,7.714286,9.133333,6.000000,8.866667,8.142858,0,0,11.25,8.25,...,0,0,0,0,0,0,0,0,0,0
6,0.0,0.714286,1.266667,1.000000,1.250000,0.857143,0,0,1.50,1.50,...,0,0,0,0,0,0,0,0,0,0
7,0.0,0.000000,0.000000,0.000000,0.066667,0.000000,0,0,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
8,1.0,2.642857,2.200000,2.333333,1.950000,3.428571,0,0,6.00,0.75,...,0,0,0,0,0,0,0,0,0,0
9,5.0,6.642857,5.666667,4.666667,5.183333,5.000000,0,1,8.50,4.50,...,0,0,0,0,0,0,0,0,0,0


In [47]:
pd.DataFrame(y_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,3.0,2.0,6.0,2.0,2.0,0.0,0.0,1.0,1.0,4.0,0.0,2.0,1.0,3.0,2.0
2,3.0,2.0,2.0,1.0,0.0,1.0,4.0,1.0,1.0,2.0,3.0,0.0,3.0,2.0,6.0,1.0
3,8.0,0.0,5.0,5.0,2.0,3.0,5.0,3.0,0.0,2.0,4.0,2.0,3.0,1.0,5.0,1.0
4,3.0,11.0,5.0,3.0,3.0,3.0,7.0,10.0,6.0,7.0,7.0,1.0,8.0,2.0,5.0,1.0
5,5.0,9.0,14.0,10.0,8.0,6.0,15.0,11.0,9.0,10.0,4.0,4.0,5.0,13.0,25.0,3.0
6,4.0,2.0,5.0,1.0,0.0,2.0,1.0,5.0,3.0,1.0,0.0,0.0,1.0,3.0,1.0,4.0
7,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
8,2.0,2.0,2.0,1.0,0.0,3.0,2.0,0.0,6.0,6.0,3.0,2.0,1.0,2.0,12.0,6.0
9,8.0,8.0,7.0,0.0,0.0,5.0,12.0,3.0,0.0,7.0,5.0,3.0,8.0,11.0,7.0,5.0


In [23]:
X_test

,day_1_2017,mean_14_2017,mean_30_2017,mean_3_2017,mean_60_2017,mean_7_2017,promo_14_2017,promo_60_2017,mean_4_dow0_2017,mean_4_dow1_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,0.0,0.571429,0.500000,0.000000,0.283333,0.142857,0,0,0.25,0.75,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.357143,0.633333,0.000000,0.966667,0.285714,0,0,1.00,0.25,...,0,0,0,0,0,0,0,0,0,0
2,0.0,1.071429,1.333333,0.333333,1.483333,0.857143,0,0,0.75,2.00,...,0,0,0,0,0,0,0,0,0,0
3,1.0,2.714286,2.466667,0.666667,2.533333,2.857143,0,0,4.00,2.75,...,0,0,0,0,0,0,0,0,0,0
4,4.0,5.071429,5.300000,2.333333,5.300000,4.714286,0,0,6.75,7.50,...,0,0,0,0,0,0,0,0,0,0
5,8.0,11.000000,10.633333,8.000000,10.200000,9.571428,0,0,9.25,11.25,...,0,0,1,1,1,1,1,1,1,1
6,2.0,0.785714,0.866667,1.000000,1.016667,0.857143,4,11,0.75,1.00,...,0,1,1,1,1,1,1,0,1,1
7,1.0,0.428571,0.400000,1.000000,0.366667,0.857143,0,7,1.25,0.25,...,0,0,0,0,0,0,0,0,1,0
8,0.0,1.642857,1.566667,0.333333,1.366667,1.142857,0,0,1.50,0.75,...,0,0,0,0,0,0,0,0,0,0
9,14.0,5.214286,4.833333,7.000000,5.083333,4.714286,0,0,6.00,5.00,...,0,0,0,0,0,0,0,0,0,0


In [31]:
print("Training and predicting models...")
params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 50
val_pred = []
test_pred = []
cate_vars = []
# Son 16 vueltas porque se van a predecir 16 días! empezando desde 2017-08-16 hasta 2017-08-31
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        # Concatenamos 6 veces items porque se eligió una ventana temporal de 6 y se concatenaron 6 X_train anteriormente.
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=20, verbose_eval=100
    )
    #Imprime los promedios deseados de "Prepare_dataset"
    #print("\n".join(("%s: %.2f" % x) for x in sorted(
        #zip(X_train.columns, bst.feature_importance("gain")),
        #key=lambda x: x[1], reverse=True
    #)))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    #Guarda la predicción de cada día
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
    
    print("Val_Pred: ",val_pred)
    print("Test_pred: ",test_pred)

#Evalua el rendimiento en un fragmento comparando las predicciones para uno de los 16 días
print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

Training and predicting models...
Step 1


/home/miguel/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 20 rounds.
Did not meet early stopping. Best iteration is:
[50]	training's l2: 225.076	valid_1's l2: 162.425
mean_7_2017: 486975681.79
mean_4_dow0_2017: 112995081.93
mean_3_2017: 100728246.06
mean_14_2017: 65462359.22
day_1_2017: 37361413.29
mean_30_2017: 31354653.53
promo_0: 14615750.33
mean_4_dow2_2017: 7016054.66
promo_60_2017: 3911399.65
promo_14_2017: 3770513.22
mean_4_dow1_2017: 3174296.15
mean_4_dow5_2017: 3079815.66
mean_4_dow6_2017: 2410827.69
mean_60_2017: 1805597.73
mean_4_dow4_2017: 1684755.52
promo_14: 862546.66
promo_7: 798946.19
mean_4_dow3_2017: 759204.06
promo_2: 103643.22
promo_9: 90027.15
promo_13: 82681.83
promo_8: 77868.30
promo_4: 54190.11
promo_6: 49355.53
promo_15: 40921.19
promo_11: 17849.57
promo_5: 13968.10
promo_1: 12920.00
promo_3: 8065.68
promo_10: 5926.96
promo_12: 0.00
Val_Pred:  [array([  0.35600261,   0.8776886 ,   1.36233583, ...,   0.40245208,
        10.76791363,   0.18744121])]
Test_pred:  [array([

Training until validation scores don't improve for 20 rounds.
Did not meet early stopping. Best iteration is:
[50]	training's l2: 228.89	valid_1's l2: 238.109
mean_60_2017: 178849852.07
mean_7_2017: 151130288.57
mean_4_dow5_2017: 143191056.82
mean_14_2017: 109681971.72
mean_30_2017: 49669773.81
mean_3_2017: 33097847.16
promo_5: 13785950.38
promo_14_2017: 10737767.33
promo_60_2017: 8566321.99
day_1_2017: 6272466.71
mean_4_dow4_2017: 4268744.13
promo_0: 3892946.67
mean_4_dow1_2017: 3089927.12
mean_4_dow0_2017: 2819516.92
mean_4_dow6_2017: 2520952.72
mean_4_dow2_2017: 2509092.05
mean_4_dow3_2017: 1765052.56
promo_4: 1283581.12
promo_7: 897389.77
promo_2: 695286.10
promo_12: 497489.90
promo_3: 473159.23
promo_6: 172132.34
promo_14: 170434.90
promo_9: 168374.30
promo_15: 95347.66
promo_11: 85352.23
promo_8: 54216.20
promo_1: 41710.30
promo_10: 23558.01
promo_13: 19235.19
Val_Pred:  [array([  0.35600261,   0.8776886 ,   1.36233583, ...,   0.40245208,
        10.76791363,   0.18744121]), arra

Val_Pred:  [array([  0.35600261,   0.8776886 ,   1.36233583, ...,   0.40245208,
        10.76791363,   0.18744121]), array([  0.45388678,   1.0288672 ,   1.49337965, ...,   0.40298004,
        11.57979065,   0.18422846]), array([  0.4857905 ,   1.11950182,   1.64093468, ...,   0.42505817,
        10.62882697,   0.25566398]), array([  0.63723182,   1.47345988,   2.2451387 , ...,   0.64855066,
        11.576782  ,   0.32637654]), array([  0.55791757,   1.38700181,   1.83183997, ...,   0.64794207,
        11.77335089,   0.40810479]), array([  0.36281957,   0.84919524,   1.53669483, ...,   0.48859142,
        11.65682041,   0.30062398]), array([ 0.31697737,  0.96442891,  1.43313626, ...,  0.38441338,
        6.23592918,  0.29229718]), array([  0.42296248,   0.9397266 ,   1.38419082, ...,   0.39729865,
        15.4972713 ,   0.30338485]), array([  0.51483066,   1.02007246,   1.30799197, ...,   0.30539092,
        13.7300582 ,   0.28949255])]
Test_pred:  [array([  0.27360798,   0.23179295,  

Training until validation scores don't improve for 20 rounds.
Did not meet early stopping. Best iteration is:
[50]	training's l2: 604.997	valid_1's l2: 140.073
mean_4_dow4_2017: 524443015.58
mean_60_2017: 330504724.28
mean_7_2017: 198306467.83
mean_3_2017: 111939458.73
mean_4_dow5_2017: 65195206.82
promo_14_2017: 19394322.66
mean_30_2017: 16865840.63
promo_11: 13225331.38
promo_60_2017: 12795962.56
mean_14_2017: 10010450.19
promo_12: 8522207.08
mean_4_dow6_2017: 5891112.18
day_1_2017: 5324452.27
mean_4_dow3_2017: 4530773.48
mean_4_dow0_2017: 4029104.55
mean_4_dow1_2017: 3756941.86
mean_4_dow2_2017: 2673287.62
promo_10: 2295721.50
promo_14: 1754818.90
promo_9: 1601002.70
promo_8: 1456541.50
promo_0: 1145667.77
promo_7: 807252.01
promo_3: 485451.51
promo_4: 478602.90
promo_2: 288785.04
promo_13: 266275.20
promo_1: 165719.70
promo_15: 145122.00
promo_5: 90101.27
promo_6: 12885.20
Val_Pred:  [array([  0.35600261,   0.8776886 ,   1.36233583, ...,   0.40245208,
        10.76791363,   0.18744

Training until validation scores don't improve for 20 rounds.
Did not meet early stopping. Best iteration is:
[50]	training's l2: 149.356	valid_1's l2: 73.2021
mean_4_dow0_2017: 243095288.66
mean_60_2017: 178745385.39
mean_7_2017: 76719542.12
mean_30_2017: 39226607.94
mean_14_2017: 29954920.46
promo_14: 27355489.03
day_1_2017: 18398969.55
mean_3_2017: 12936672.14
mean_4_dow2_2017: 11036568.78
promo_60_2017: 5974974.86
promo_14_2017: 5350616.90
mean_4_dow1_2017: 3665965.46
mean_4_dow6_2017: 3391222.82
promo_7: 2845666.45
mean_4_dow5_2017: 2484488.41
mean_4_dow3_2017: 2441816.24
mean_4_dow4_2017: 2023298.81
promo_12: 1007114.44
promo_0: 996442.59
promo_11: 994973.80
promo_9: 766385.09
promo_15: 534180.59
promo_8: 457702.70
promo_13: 208884.97
promo_4: 144306.03
promo_3: 121525.40
promo_2: 98369.39
promo_1: 70492.20
promo_10: 70227.84
promo_6: 65954.60
promo_5: 25764.08
Val_Pred:  [array([  0.35600261,   0.8776886 ,   1.36233583, ...,   0.40245208,
        10.76791363,   0.18744121]), arr

In [39]:
pd.DataFrame(np.array(val_pred).transpose())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.356003,0.453887,0.485791,0.637232,0.557918,0.362820,0.316977,0.422962,0.514831,0.565326,0.598180,0.630977,0.416385,0.455010,0.399595,0.439131
1,0.877689,1.028867,1.119502,1.473460,1.387002,0.849195,0.964429,0.939727,1.020072,1.192140,1.530023,1.293385,1.006039,0.941444,0.990264,1.003619
2,1.362336,1.493380,1.640935,2.245139,1.831840,1.536695,1.433136,1.384191,1.307992,1.560269,2.012137,1.573342,1.474204,1.517831,1.304439,1.327151
3,2.144002,1.967369,2.354353,3.365954,2.718954,2.198546,2.046316,2.101807,1.854168,2.246714,3.224667,2.170942,2.160797,2.025969,1.961152,1.882214
4,5.202675,4.801131,5.400824,6.064102,5.260626,5.568464,4.952917,4.656863,4.294023,4.967596,5.952821,4.665657,5.430520,4.623407,4.660452,4.093617
5,10.668632,8.208584,9.071405,10.734254,7.838103,9.684129,9.295849,9.512131,7.796337,8.801114,10.053687,7.277471,9.474867,8.793620,8.771965,7.821329
6,0.907783,0.851580,0.951850,3.493203,1.449163,0.738331,0.754424,0.717984,2.544366,1.137700,1.699753,3.945217,3.733647,1.081309,2.939300,0.987366
7,2.149335,0.219298,0.177346,0.656868,0.432931,0.343637,0.348457,0.169014,0.366527,0.221949,0.374831,0.532118,0.364643,0.331229,0.338763,0.362111
8,1.410655,1.407439,1.523877,2.253036,1.831840,1.787622,1.631321,1.409494,1.279449,1.431167,1.937154,1.573342,1.571741,1.400471,1.472024,1.134591
9,3.500813,3.857103,3.642076,4.906644,4.032294,3.881020,3.769116,3.798710,3.379644,3.579172,4.587904,3.959448,4.258972,3.864271,3.637619,3.351362


In [43]:
print("Making submission...")
y_test = np.array(test_pred).transpose()

Making submission...


In [51]:
#Predicciones para cada tienda, item en cada uno de los 16 días
pd.DataFrame(y_test, index=df_2017.index,columns=pd.date_range("2017-08-16", periods=16))

2017-08-16  2017-08-17  2017-08-18  2017-08-19  \
store_nbr item_nbr                                                   
1         96995       0.273608    0.327214    0.442796    0.555550   
          99197       0.231793    0.321843    0.463700    0.526977   
          103520      1.003762    0.847976    1.001632    1.412170   
          103665      2.450797    2.069898    2.467543    2.794524   
          105574      4.990742    4.535217    4.791298    5.756059   
          105575      9.360047    8.983312    9.445489   11.167188   
          105577      0.329212    0.459341    0.438936    1.093148   
          105693      0.774119    0.635369    0.623140    0.832983   
          105737      1.113081    1.229717    1.292964    1.984636   
          105857      4.896538    4.117029    4.527484    5.599566   
          106716      2.474053    2.245063    2.458344    3.223825   
          108079      0.718574    0.641361    0.724447    0.937895   
          108634      0.187441    0.184228    0.255664    0.326377   
          108696      1.622190    1.606291    2.815293    2.367576   
          108698      0.713756    0.855299    2.039293    1.111505   
          108701      2.212867    0.825970    0.793282    1.464394   
          108786      2.193222    2.006759    2.473383    2.735232   
          108797      3.857870    3.621205    4.218584    4.790437   
          108862      0.753768    0.833170    0.866492    1.379931   
          108952      1.766856    1.585973    1.412174    2.319742   
          111223      5.827152    4.719242    4.815293    7.046702   
          111397      0.605245    0.571980    0.737016    0.972450   
          112830      2.210773    2.136166    2.325474    2.789718   
          114778      2.464326    2.220056    2.782957    3.401914   
          114790      3.752702    3.119058    3.376134    2.739950   
          114799      0.904081    0.920777    0.933421    1.290760   
          114800      2.462627    2.321212    3.153295    3.561534   
          115267      1.764365    1.850469    2.684729    2.954330   
          115611      5.670872    5.369950    6.431251    7.398063   
          115693      2.259511    2.000114    2.327693    2.736800   
...                        ...         ...         ...         ...   
54        2081056     2.155411    1.693212    1.922001    2.590937   
          2081064    47.881004   44.855572   43.252057   59.085811   
          2081095     2.550122    2.320880    2.784620    3.379328   
          2081142     0.402452    0.428398    0.606969    0.635491   
          2081161    21.456418   19.469220   23.006040   29.386501   
          2083837     2.234967    1.930840    2.686670    2.895531   
          2083859     1.858486    1.497272    1.398885    3.188971   
          2084278     2.541434    2.109769    2.734495    3.637532   
          2084557     7.620296    7.011403    7.961491   13.231469   
          2086882     1.669321    1.868734    3.596060    3.133447   
          2087374     2.497227    1.307168    1.203647    2.113485   
          2087409     1.039852    0.878036    0.946596    1.171974   
          2087543     0.479485    0.832777    0.781166    1.184878   
          2087567     3.094342    1.360768    1.935792    1.909796   
          2087933     0.187441    0.184228    0.302355    0.351702   
          2087978     7.431224    5.780715    9.907542    9.058446   
          2088922     7.922333    6.808080    7.941440   12.574451   
          2089036     3.334085    3.205458    3.413543    5.093619   
          2089339     3.404911    3.726312    4.204811    5.406745   
          2099230     0.187441    0.184228    0.255664    0.290222   
          2101795     2.025024    0.184228    0.149133    0.142048   
          2103250     0.189991    0.167939    0.238368    0.290222   
          2105347     0.255131    0.279102    0.342218    0.472869   
          2106464     2.140441    0.860000    0.663691    2.564476   
          2108569     1.184081

In [ ]:
print("Making submission...")
y_test = np.array(test_pred).transpose()

df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

# Une las predicciones con el test (basandose en tienda, item y fecha) y los que no estén entonces lo llena con 0
submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)

In [3]:
from dask.distributed import Client
import dask.dataframe as dd

# Lanzo el Cliente y ejecuto el Dashboard

client = Client()
client

Client Scheduler: tcp://127.0.0.1:44009 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 8.36 GB


## Data Exploration

In [3]:
import dask.dataframe as dd

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32', 'onpromotion':'bool' }
df_train = dd.read_csv('data/train.csv', dtype=dtypes, parse_dates=["date"], converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0}, usecols=[1, 2, 3, 4, 5])
df_train.head(5)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [ ]:
import dask.dataframe as dd
import numpy as np

dtypes = {'item_nbr':'int32', 'store_nbr':'int8', 'onpromotion':'bool' }
df_train = dd.read_csv('data/train.csv', dtype=dtypes, parse_dates=["date"], converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0}, usecols=[1, 2, 3, 4, 5]).compute()
df_train.head(5)

In [ ]:
df_train["unit_sales"]

In [2]:
df_test = dd.read_csv('data/test.csv')
df_test.head(5)

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


In [6]:
df_sample_submission = dd.read_csv('data/sample_submission.csv')
df_sample_submission.head(5)

,id,unit_sales
0,125497040,0
1,125497041,0
2,125497042,0
3,125497043,0
4,125497044,0


In [5]:
df_stores = dd.read_csv('data/stores.csv')
df_stores.head(5)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [12]:
df_items = dd.read_csv('data/items.csv')
df_items.head(5)

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [17]:
df_oil = dd.read_csv('data/oil.csv')
df_oil.head(5)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [19]:
df_holidays_events = dd.read_csv('data/holidays_events.csv')
df_holidays_events.head(5)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [47]:
df_transactions = dd.read_csv('data/transactions.csv')
df_transactions.head(5)

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


## Porcent of Null Values

In [39]:
def null_values(df):
    start = time.time()
    total = len(df)
    result = (df.isnull().sum()/total).compute()
    print("Lenght of df: ",total)
    print("Time: ", time.strftime("%M:%S", time.gmtime(time.time() - start)) )
    return result

In [41]:
null_values(df_train)

Lenght of df:  125497040
Time:  04:38


id             0.000000
date           0.000000
store_nbr      0.000000
item_nbr       0.000000
unit_sales     0.000000
onpromotion    0.172575
dtype: float64

In [42]:
null_values(df_test)

Lenght of df:  3370464
Time:  00:06


id             0.0
date           0.0
store_nbr      0.0
item_nbr       0.0
onpromotion    0.0
dtype: float64

In [43]:
null_values(df_stores)

Lenght of df:  54
Time:  00:00


store_nbr    0.0
city         0.0
state        0.0
type         0.0
cluster      0.0
dtype: float64

In [44]:
null_values(df_items)

Lenght of df:  4100
Time:  00:00


item_nbr      0.0
family        0.0
class         0.0
perishable    0.0
dtype: float64

In [48]:
null_values(df_transactions)

Lenght of df:  83488
Time:  00:00


date            0.0
store_nbr       0.0
transactions    0.0
dtype: float64

In [49]:
null_values(df_oil)

Lenght of df:  1218
Time:  00:00


date          0.000000
dcoilwtico    0.035304
dtype: float64

In [51]:
null_values(df_holidays_events)

Lenght of df:  350
Time:  00:00


date           0.0
type           0.0
locale         0.0
locale_name    0.0
description    0.0
transferred    0.0
dtype: float64

In [54]:
null_values(df_sample_submission)

Lenght of df:  3370464
Time:  00:02


id            0.0
unit_sales    0.0
dtype: float64

In [8]:
import dask.dataframe as dd
df_train = dd.read_csv('data/train.csv', parse_dates=["date"])
df_train.head(5)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [ ]:
#df_train['unit_sales'].apply(lambda x: np.log1p(x), meta={'unit_sales': 'int64'}) #logarithm conversion
#df_train['date'].dt.dayofweek.compute()

In [ ]:
#df_train_2.dropna().sort_index(axis=0, level=2, ascending=True)

In [20]:
## MI SOLUCIÓN ######

import pandas as pd
import numpy as np
import time

start = time.time()

train_list = []

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32', 'onpromotion':'bool' }

chunk_iter = pd.read_csv("data/train.csv", dtype=dtypes, parse_dates=["date"], chunksize=10000)

for chunk in chunk_iter:
    chunk_kept = chunk[(chunk["date"].dt.year == 2017)]
    train_list.append(chunk_kept)
df_train_2 = pd.concat(train_list)

print("Time: ", time.strftime("%M:%S", time.gmtime(time.time() - start)) )

Time:  03:53


In [ ]:
df_train_2.head()

In [21]:
df_train_2.loc[(df_train_2.unit_sales < 0),'unit_sales'] = 0 # Eliminar Valores Negativos
df_train_2['unit_sales'] =  df_train_2['unit_sales'].apply(pd.np.log1p) #Calcular Logaritmo de Unidades
df_train_2['dow'] = df_train_2['date'].dt.dayofweek

#Promedio de Ventas Diarias
ma_dw = df_train_2.groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw').reset_index()

#Promedio de Ventas Semanales
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(
       ['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk').reset_index()

# Eliminar columnas de id, promociones y Dia de la Semana
df_train_2 = df_train_2.drop(['id','onpromotion','dow'],axis=1)

# Se crean registros para todos los items, todas las Tiendas en todas las Fechas
# para el correcto calculo del promedio de ventas diarias

# Busca valores únicos de Items, Tiendas, Fechas
u_dates = df_train_2.date.unique()
u_stores = df_train_2.store_nbr.unique()
u_items = df_train_2.item_nbr.unique()

# Hace index por Fecha, Tienda, Item
df_train_2.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)

df_train_2 = df_train_2.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
).reset_index()

# Rellenar con NaN
df_train_2['unit_sales'].fillna(0, inplace=True)

# Toma la Última Fecha
lastdate = df_train_2.iloc[df_train_2.shape[0]-1].date

# Calcula el promedio de ventas por item y tienda
ma_is = df_train_2[['item_nbr','store_nbr','unit_sales']].groupby(
        ['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais')

In [56]:
from datetime import timedelta

#Moving Averages
for i in [112,56,28,14,7,3,1]:
    tmp = df_train_2[df_train_2.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

In [57]:
ma_is

mais   mais112    mais56    mais28    mais14  \
item_nbr store_nbr                                                     
99197    1          0.000000  0.000000  0.000000  0.000000  0.000000   
         2          0.346574  0.346574  0.346574  0.346574  0.346574   
         3          0.804719  0.804719  0.804719  0.804719  0.804719   
         4          0.346574  0.346574  0.346574  0.346574  0.346574   
         5          0.000000  0.000000  0.000000  0.000000  0.000000   
         6          0.346574  0.346574  0.346574  0.346574  0.346574   
         7          0.804719  0.804719  0.804719  0.804719  0.804719   
         8          0.693147  0.693147  0.693147  0.693147  0.693147   
         9          0.000000  0.000000  0.000000  0.000000  0.000000   
         10         0.000000  0.000000  0.000000  0.000000  0.000000   
         11         0.000000  0.000000  0.000000  0.000000  0.000000   
         12         0.000000  0.000000  0.000000  0.000000  0.000000   
         13         0.000000  0.000000  0.000000  0.000000  0.000000   
         14         0.000000  0.000000  0.000000  0.000000  0.000000   
         15         0.000000  0.000000  0.000000  0.000000  0.000000   
         16         0.000000  0.000000  0.000000  0.000000  0.000000   
         17         0.000000  0.000000  0.000000  0.000000  0.000000   
         18         0.000000  0.000000  0.000000  0.000000  0.000000   
         19         0.000000  0.000000  0.000000  0.000000  0.000000   
         20         0.000000  0.000000  0.000000  0.000000  0.000000   
         21         0.000000  0.000000  0.000000  0.000000  0.000000   
         22         0.000000  0.000000  0.000000  0.000000  0.000000   
         23         0.549306  0.549306  0.549306  0.549306  0.549306   
         25         0.346574  0.346574  0.346574  0.346574  0.346574   
103501   1          0.000000  0.000000  0.000000  0.000000  0.000000   
         2          0.000000  0.000000  0.000000  0.000000  0.000000   
         3          0.000000  0.000000  0.000000  0.000000  0.000000   
         4          0.000000  0.000000  0.000000  0.000000  0.000000   
         5          0.000000  0.000000  0.000000  0.000000  0.000000   
         6          0.000000  0.000000  0.000000  0.000000  0.000000   
...                      ...       ...       ...       ...       ...   
2057231  19         0.000000  0.000000  0.000000  0.000000  0.000000   
         20         1.481914  1.481914  1.481914  1.481914  1.481914   
         21         1.220042  1.220042  1.220042  1.220042  1.220042   
         22         0.000000  0.000000  0.000000  0.000000  0.000000   
         23         0.885183  0.885183  0.885183  0.885183  0.885183   
         25         1.477429  1.477429  1.477429  1.477429  1.477429   
2058907  1          0.000000  0.000000  0.000000  0.000000  0.000000   
         2          0.804719  0.804719  0.804719  0.804719  0.804719   
         3          0.000000  0.000000  0.000000  0.000000  0.000000   
         4          0.000000  0.000000  0.000000  0.000000  0.000000   
         5          0.346574  0.346574  0.346574  0.346574  0.346574   
         6          0.000000  0.000000  0.000000  0.000000  0.000000   
         7          0.000000  0.000000  0.000000  0.000000  0.000000   
         8          0.549306  0.549306  0.549306  0.549306  0.549306   
         9          0.000000  0.000000  0.000000  0.000000  0.000000   
         10         0.000000  0.000000  0.000000  0.000000  0.000000   
         11         0.000000  0.000000  0.000000  0.000000  0.000000   
         12         0.000000  0.000000  0.000000  0.000000  0.000000   
         13         0.000000  0.000000  0.000000  0.000000  0.000000   
         14         0.000000  0.000000  0.000000  0.000000  0.000000   
         15         0.000000  0.000000  0.000000  0.000000  0.000000   
         16         0.000000  0.000000  0.000000  0.000000  0.000000   
         17         0.000000  0.000000  0.000000  0.000000  0.00

In [ ]:


del tmp,tmpg,train

ma_is['mais']=ma_is.median(axis=1)
ma_is.reset_index(inplace=True)
ma_is.drop(list(ma_is.columns.values)[3:],1,inplace=True)

## SOLUCION KAGGLE

In [3]:
import pandas as pd
import numpy as np
import time

start = time.time()

train_list = []

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}

chunk_iter = pd.read_csv("data/train.csv", usecols=[1,2,3,4], dtype=dtypes, parse_dates=["date"], skiprows=range(1, 86672217) , chunksize=1000000)

for chunk in chunk_iter:
    train_list.append(chunk)
df_train = pd.concat(train_list)

print("Time: ", time.strftime("%M:%S", time.gmtime(time.time() - start)) )
df_train.head()

Time:  01:54


,date,store_nbr,item_nbr,unit_sales
0,2016-08-01,1,103520,3.0
1,2016-08-01,1,103665,2.0
2,2016-08-01,1,105574,7.0
3,2016-08-01,1,105575,13.0
4,2016-08-01,1,105577,2.0


In [7]:
df_train.loc[(df_train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
df_train['unit_sales'] =  df_train['unit_sales'].apply(pd.np.log1p) #logarithm conversion

In [8]:
df_train['dow'] = df_train['date'].dt.dayofweek

In [19]:
#Days of Week Means
ma_dw = df_train[['item_nbr','store_nbr','dow','unit_sales']].groupby(
       ['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw').reset_index()

In [30]:
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(
       ['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk').reset_index()

In [33]:
df_train.drop('dow',1,inplace=True)

In [ ]:
# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.

# Busca valores únicos de Items, Tiendas, Fechas

u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()

# Hace index por Fecha, Tienda, Item
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)

# Reindexa en base a valores únicos de Items, Tiendas y Fechas
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
).reset_index()

del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
lastdate = train.iloc[train.shape[0]-1].date

In [24]:
df_test = pd.read_csv("data/test.csv")
df_test["onpromotion"] = df_test["onpromotion"]*1
df_test.head()

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,0
1,125497041,2017-08-16,1,99197,0
2,125497042,2017-08-16,1,103501,0
3,125497043,2017-08-16,1,103520,0
4,125497044,2017-08-16,1,103665,0


In [25]:
df_train["onpromotion"] = df_train["onpromotion"]*1
df_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
101688779,101688779,2017-01-01,25,99197,1.0,0
101688780,101688780,2017-01-01,25,103665,7.0,0
101688781,101688781,2017-01-01,25,105574,1.0,0
101688782,101688782,2017-01-01,25,105857,4.0,0
101688783,101688783,2017-01-01,25,106716,2.0,0


In [26]:
df_train = df_train.head(500000)

In [27]:
df_train_2 = df_train

In [29]:
df_train_2["date"] = df_train_2["date"].dt.dayofweek
df_train_2.groupby(["item_nbr","store_nbr","date"]).mean()

id  unit_sales
item_nbr store_nbr date                       
99197    1         1     101804697         3.0
                   2     101914704         1.0
                   3     102021351         1.0
                   4     102123566         1.0
         2         0     101691960         1.0
                   2     101916926         1.0
         3         0     101694535         4.0
                   1     101809388         1.0
                   2     101919295         2.0
                   3     102025767         5.0
                   4     102128116         1.0
         4         0     101697392         1.0
                   1     101812075         1.0
                   4     102130879         1.0
         6         0     101702047         1.0
                   1     101816576         2.0
         7         0     101704702         4.0
                   1     101819103         5.0
                   2     101928888         2.0
                   3     102034887         3.0
                   4     102137650         2.0
         8         0     101707148         3.0
                   1     101821531         3.0
                   2     101931293         4.0
                   3     102037188         2.0
         23        0     101736722         2.0
         25        1     101853597         2.0
                   2     101962201         4.0
                   3     102067019         2.0
                   6     101688779         1.0
...                            ...         ...
2058907  2         4     102128115         1.0
         3         1     101812074         2.0
         5         0     101702046         1.0
                   1     101816575         1.0
                   2     101926354         2.0
                   4     102135210         1.0
         7         1     101821530         2.0
                   3     102037187         1.0
         8         0     101709982         2.0
                   1     101824174         1.0
                   3     102039790         1.0
         24        1     101853596         1.0
                   2     101962200         1.0
                   3     102067018         2.0
                   4     102169057         2.0
         37        2     101984228         1.0
         41        1     101885194         1.0
                   2     101992445         2.0
                   3     102095699         2.0
         45        0     101784047         1.0
                   2     102002242         2.0
                   3     102105071         1.0
         47        2     102007716         1.0
                   3     102110355         1.0
         49        1     101906199         2.0
         50        0     101798504         2.0
         51        0     101800952         1.0
                   1     101911160         3.0
                   2     102017950         2.0
                   3     102120240         1.0

[500000 rows x 2 columns]

In [ ]:
import pandas as pd
from datetime import timedelta

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}

train = pd.read_csv('../input/train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 86672217) #Skip dates before 2016-08-01
                    )

train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(
        ['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw').reset_index()
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(
        ['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk').reset_index()

train.drop('dow',1,inplace=True)

In [54]:
#train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train = df_train
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train["date"]#.dt.dayofweek

#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(
       ['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw').reset_index()
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(
      ['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')#.reset_index()

ma_dw.head()

,item_nbr,store_nbr,dow,madw
0,99197,1,1,0.287938
1,99197,1,2,0.234307
2,99197,1,3,0.234307
3,99197,1,4,0.234307
4,99197,2,0,0.234307


In [53]:
ma_wk.head()

mawk
store_nbr item_nbr          
1         99197     0.281443
          103520    0.305186
          103665    0.318904
          105574    0.370282
          105575    0.378390

In [14]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


X = df_train_.head(500000).drop(['unit_sales','date'], axis=1).fillna(0)
y = df_train_.head(500000)['unit_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

estimator = DecisionTreeRegressor()
estimator.fit(X_train, y_train)
y_predict = estimator.predict(X_test)

mean_squared_error(y_test,y_predict)

424.49312218443998

In [43]:
df_solution = pd.DataFrame(np.column_stack([df_test["id"],predicciones]), columns=["id","unit_sales"])

df_solution.loc[(df_solution.unit_sales < 0),'unit_sales'] = 0

pd.concat([df_solution["id"].astype(int),df_solution["unit_sales"]], axis=1).to_csv("predictions_11.csv", float_format='%.4f', index=None)

In [ ]:
weight = items["perishable"] * 0.25 + 1

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
	assert len(y) == len(y_pred)
	terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
	return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

def rmsle(y_true,y_pred):
   terms = list()
   for i in range(len(y_pred)):
       terms.append( np.square(  np.log(y_pred[i] + 1) - np.log(y_true[i] + 1)  ) )
   return ( np.sum( weight * np.sum(terms) ) / weight.sum() ) ** 0.5

rmsle(y_test.values,y_predict)